# How to download the union of gobjaverse and lvis

Download the annotations files first.

In [ ]:
from huggingface_hub import hf_hub_download

# 'gobjaverse_280k_index_to_objaverse.json' is provided by the official.
hf_hub_download(repo_id="alexzyqi/GPT4Point", filename='gobjaverse_280k_index_to_objaverse.json', repo_type="dataset", local_dir='annotations/280k_index_to_objaverse/')

# objaverse-lvis annotations
hf_hub_download(repo_id="alexzyqi/GPT4Point", filename='ann_lvis.json', repo_type="dataset", local_dir='annotations/')

#### Step 1: get the gobjaverse to objaverse.

In [ ]:
import json

# 'gobjaverse_280k_index_to_objaverse.json' is provided by the official.
with open('annotations/280k_index_to_objaverse/gobjaverse_280k_index_to_objaverse.json', 'r') as index_file:
    index_dict = json.load(index_file)

swapped_dict = {v.split("/")[-1].split(".")[0]: k for k, v in index_dict.items()}

# 'reverse_280k_to_objaverse.json': 
with open('annotations/280k_index_to_objaverse/objaverse_280k_index_to_gobjaverse.json', 'w') as swapped_file:
    json.dump(swapped_dict, swapped_file, indent=2)

#### Step 2: select the 280k_lvis(dict format).

In [ ]:
with open('annotations/ann_lvis.json', 'r') as ann_file, open('annotations/280k_index_to_objaverse/objaverse_280k_index_to_gobjaverse.json', 'r') as index_file:
    ann_data = json.load(ann_file)
    index_data = json.load(index_file)

lvis_280k_dict = {}

for category, id_list in ann_data.items():
    for obj_id in id_list:
        if obj_id in index_data:
           lvis_280k_dict[obj_id] = index_data[obj_id]

lvis_280k_list = list(lvis_280k_dict.values())

with open('annotations/download_280k_lvis.json', 'w') as outfile:
    json.dump(lvis_280k_list, outfile)

#### Optional: split the 280k_lvis to 10 splits for download.

In [ ]:
with open('annotations/download_280k_lvis.json', 'r') as lvis_280k_file:
    original_list = json.load(lvis_280k_file)

chunk_size = len(original_list) // 10 

result_chunks = [original_list[i:i+chunk_size] for i in range(0, len(original_list), chunk_size)]
for i, chunk in enumerate(result_chunks):
    filename = f"annotations/split_280k_lvis/download_280k_lvis_{i+1}.json"
    with open(filename, 'w') as json_file:
        json.dump(chunk, json_file)